In [8]:
from datetime import date
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 5)

df = catalog.load('raw/openaire/researchproduct_dev#parquet')

[10/15/25 20:59:04] INFO     Loading data from raw/openaire/researchproduct_dev#parquet         ]8;id=499903;file:///home/pablo/dev/conicet/kedro-conicet/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=863462;file:///home/pablo/dev/conicet/kedro-conicet/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (ParquetDataset)...                                                                   

In [9]:
df

,authors,openAccessColor,publiclyFunded,type,language,countries,subjects,mainTitle,subTitle,descriptions,publicationDate,publisher,embargoEndDate,sources,formats,contributors,coverages,bestAccessRight,container,documentationUrls,codeRepositoryUrl,programmingLanguage,contactPeople,contactGroups,tools,size,version,geoLocations,id,originalIds,pids,dateOfCollection,lastUpdateTimeStamp,indicators,projects,organizations,communities,collectedFrom,instances,isGreen,isInDiamondJournal,relOrganizationId,load_datetime
0,"[{'fullName': 'Susevich, María Laura', 'name':...",gold,False,publication,"{'code': 'spa', 'label': 'Spanish; Castilian'}",None,"[{'provenance': None, 'subject': {'scheme': 'k...",Dicistrovirus from the pollinator community fo...,None,[Los Dicistrovirus son una familia de virus qu...,2022-08-31,"Universidad Nacional Mayor de San Marcos, Facu...",None,[Revista de Investigaciones Veterinarias del P...,[application/pdf],None,None,"{'code': 'c_abf2', 'label': 'OPEN', 'scheme': ...","{'conferenceDate': None, 'conferencePlace': No...",None,None,None,None,None,None,None,None,None,4dc99724cf04::319dc88111c9b2d6021228590e79130a,[50|4dc99724cf04::319dc88111c9b2d6021228590e79...,None,None,None,"{'citationImpact': {'citationClass': 'C5', 'ci...",None,"[{'acronym': 'UNLPAM', 'id': 'openorgs____::db...","[{'code': 'knowmad', 'label': 'Knowmad Institu...",[{'key': 'openaire____::0b74b6a356bbf23c245f9a...,"[{'accessRight': {'code': 'c_abf2', 'label': '...",False,False,openorgs____::40b9f835648a3e0d057d6917dd7e54d5,2025-09-04
1,"[{'fullName': 'Steffen, Kevin Denis', 'name': ...",gold,False,publication,"{'code': 'spa', 'label': 'Spanish; Castilian'}",None,"[{'provenance': None, 'subject': {'scheme': 'k...",Variability in the growth rates of Saanen kids...,None,[This study was carried out to determine the v...,2023-12-18,"Universidad Nacional Mayor de San Marcos, Facu...",None,[Revista de Investigaciones Veterinarias del P...,[application/pdf],None,None,"{'code': 'c_abf2', 'label': 'OPEN', 'scheme': ...","{'conferenceDate': None, 'conferencePlace': No...",None,None,None,None,None,None,None,None,None,4dc99724cf04::95ea5df70a451a0487e051faa6c0a646,"[oai:ojs.csi.unmsm:article/25240, 50|4dc99724c...",None,None,None,"{'citationImpact': {'citationClass': 'C5', 'ci...",None,[{'acronym': 'Universidad Nacional de La Pampa...,None,[{'key': 'openaire____::0b74b6a356bbf23c245f9a...,"[{'accessRight': {'code': 'c_abf2', 'label': '...",False,False,openorgs____::40b9f835648a3e0d057d6917dd7e54d5,2025-09-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,"[{'fullName': 'Seré, Pablo Ricardo', 'name': '...",None,False,publication,"{'code': 'esl/spa', 'label': 'Spanish'}","[{'code': 'AR', 'label': 'Argentina', 'provena...","[{'provenance': None, 'subject': {'scheme': 'k...",Efecto de la concentración del silano MTMO sob...,None,[Los pretratamientos superficiales a base de s...,2017-01-01,None,None,"[reponame:SEDICI (UNLP), instname:Universidad ...",[application/pdf],None,None,"{'code': 'c_abf2', 'label': 'OPEN', 'scheme': ...",None,None,None,None,None,None,None,None,None,None,dedup_wf_002::00cece367574590aa05deb123340c6e6,"[oai:sedici.unlp.edu.ar:10915/94520, 50|od____...",None,None,None,"{'citationImpact': {'citationClass': 'C5', 'ci...",None,"[{'acronym': 'UNLP', 'id': 'openorgs____::40b9...",None,[{'key': 'opendoar____::dc6e224a8d74ce03bf3011...,"[{'accessRight': None, 'alternateIdentifiers':...",True,False,openorgs____::40b9f835648a3e0d057d6917dd7e54d5,2025-09-04
299,"[{'fullName': 'TomÃ¡s, Facundo', 'name': 'Facu...",gold,False,publication,"{'code': 'eng', 'label': 'English'}","[{'code': 'AR', 'label': 'Argentina', 'provena...","[{'provenance': None, 'subject': {'scheme': 'k...",Guerra Civil española y carteles de propaganda...,None,[El cartelismo durante la Guerra Civil español...,2006-01-01,Universidad Nacional de La Plata,

In [10]:
def _pick_load_dt(df: pd.DataFrame):
    # Si hay una sola fecha en el batch, usala; si hay varias, quedate con la más reciente;
    # si no hay, hoy.
    if 'load_datetime' not in df.columns or df['load_datetime'].isna().all():
        return date.today()
    vals = df['load_datetime'].dropna()
    if vals.nunique() == 1:
        return vals.iloc[0]
    return pd.to_datetime(vals).max().date()

# Transformaciones

In [11]:
df_research_collectedfrom = df[['id','collectedFrom']].explode('collectedFrom').reset_index(drop=True)
df_research_collectedfrom.rename(columns={'id':'researchproduct_id'}, inplace=True)

In [12]:
df_research_collectedfrom

,researchproduct_id,collectedFrom
0,4dc99724cf04::319dc88111c9b2d6021228590e79130a,{'key': 'openaire____::0b74b6a356bbf23c245f9ae...
1,4dc99724cf04::95ea5df70a451a0487e051faa6c0a646,{'key': 'openaire____::0b74b6a356bbf23c245f9ae...
...,...,...
641,dedup_wf_002::00cf1fe88397a2c3ce9d41fe616b29cf,{'key': 'driver______::bee53aa31dc2cbb538c10c2...
642,dedup_wf_002::00cf1fe88397a2c3ce9d41fe616b29cf,{'key': 'opendoar____::01e9565cecc4e989123f962...


In [13]:
df_research_collectedfrom['collectedFrom'] \
    .dropna() \
    .map(lambda d: sorted(d.keys())) \
    .explode() \
    .value_counts()


collectedFrom
key      643
value    643
Name: count, dtype: int64

In [14]:
df_collectedfrom = pd.json_normalize(df_research_collectedfrom['collectedFrom'])
df_collectedfrom.rename(columns={'key':'datasource_id'}, inplace=True)

In [15]:
df_collectedfrom

,datasource_id,value
0,openaire____::0b74b6a356bbf23c245f9ae9a748745c,Revistas de investigación Universidad Nacional...
1,openaire____::0b74b6a356bbf23c245f9ae9a748745c,Revistas de investigación Universidad Nacional...
...,...,...
641,driver______::bee53aa31dc2cbb538c10c2b65fa5824,DOAJ
642,opendoar____::01e9565cecc4e989123f9620c1d09c09,Servicio de Difusión de la Creación Intelectual


## Paso 1: Convierto tipos y selecciono columnas con cardinalidad 1 con respecto a cada research product
+ info en https://graph.openaire.eu/docs/data-model/entities/research-product

In [16]:
def openaire_land_researchproduct_collectedfrom(df: pd.DataFrame)-> pd.DataFrame:

    load_dt = _pick_load_dt(df)

    df_research_collectedfrom = df[['id','collectedFrom']].explode('collectedFrom').reset_index(drop=True)
    df_research_collectedfrom.rename(columns={'id':'researchproduct_id'}, inplace=True)

    df_collectedfrom = pd.json_normalize(df_research_collectedfrom['collectedFrom'])
    df_collectedfrom.rename(columns={'key':'datasource_id'}, inplace=True)

    df_research_collectedfrom = pd.concat(
        [df_research_collectedfrom['researchproduct_id'], df_collectedfrom.loc[:,['datasource_id','value']]], 
        axis=1
    )

    df_research_collectedfrom['load_datetime'] = load_dt

    return df_research_collectedfrom


In [17]:
df_research_collectedfrom = openaire_land_researchproduct_collectedfrom(df)

In [18]:
df_research_collectedfrom

,researchproduct_id,datasource_id,value,load_datetime
0,4dc99724cf04::319dc88111c9b2d6021228590e79130a,openaire____::0b74b6a356bbf23c245f9ae9a748745c,Revistas de investigación Universidad Nacional...,2025-09-04
1,4dc99724cf04::95ea5df70a451a0487e051faa6c0a646,openaire____::0b74b6a356bbf23c245f9ae9a748745c,Revistas de investigación Universidad Nacional...,2025-09-04
...,...,...,...,...
641,dedup_wf_002::00cf1fe88397a2c3ce9d41fe616b29cf,driver______::bee53aa31dc2cbb538c10c2b65fa5824,DOAJ,2025-09-04
642,dedup_wf_002::00cf1fe88397a2c3ce9d41fe616b29cf,opendoar____::01e9565cecc4e989123f9620c1d09c09,Servicio de Difusión de la Creación Intelectual,2025-09-04
